In [1]:
from os import getenv
import polars as pl
from dotenv import load_dotenv
from neo4j import GraphDatabase

In [2]:
load_dotenv()

True

#### Credit to Chidambara Raju G for data
[Link](https://www.kaggle.com/datasets/rajugc/imdb-top-250-movies-dataset)

In [3]:
df = pl.read_csv("../data/data.csv")

#### Removing and renaming columns

In [4]:
df.columns

['rank',
 'name',
 'year',
 'rating',
 'genre',
 'certificate',
 'run_time',
 'tagline',
 'budget',
 'box_office',
 'casts',
 'directors',
 'writers']

In [5]:
drop_cols = [ "rank", "whereToRent","availableStreamingPlatforms", "budget", "box_office"]
df = df.drop(drop_cols).rename({"name": "title"})
df.columns

['title',
 'year',
 'rating',
 'genre',
 'certificate',
 'run_time',
 'tagline',
 'casts',
 'directors',
 'writers']

In [6]:
df.describe()

statistic,title,year,rating,genre,certificate,run_time,tagline,casts,directors,writers
str,str,f64,f64,str,str,str,str,str,str,str
"""count""","""250""",250.0,250.0,"""250""","""250""","""250""","""250""","""250""","""250""","""250"""
"""null_count""","""0""",0.0,0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,1986.36,8.3072,null,null,null,null,null,null,null
"""std""",null,25.125356,0.229081,null,null,null,null,null,null,null
"""min""","""12 Angry Men""",1921.0,8.0,"""Action,Adventure""","""13+""","""1h 20m""","""""As far back as I can remember…","""Aamir Khan,Madhavan,Mona Singh…","""Aamir Khan,Amole Gupte(uncredi…","""Aaron Guzikowski"""
"""25%""",null,1966.0,8.1,null,null,null,null,null,null,null
"""50%""",null,1994.0,8.2,null,null,null,null,null,null,null
"""75%""",null,2006.0,8.4,null,null,null,null,null,null,null
"""max""","""Your Name.""",2022.0,9.3,"""Western""","""X""","""Not Available""","""ventureComedyCrime""","""Çetin Tekindor,Fikret Kuskan,H…","""Çagan Irmak""","""Çagan Irmak"""


In [7]:
df.head()

title,year,rating,genre,certificate,run_time,tagline,casts,directors,writers
str,i64,f64,str,str,str,str,str,str,str
"""The Shawshank Redemption""",1994,9.3,"""Drama""","""R""","""2h 22m""","""Fear can hold you prisoner. Ho…","""Tim Robbins,Morgan Freeman,Bob…","""Frank Darabont""","""Stephen King,Frank Darabont"""
"""The Godfather""",1972,9.2,"""Crime,Drama""","""R""","""2h 55m""","""An offer you can't refuse.""","""Marlon Brando,Al Pacino,James …","""Francis Ford Coppola""","""Mario Puzo,Francis Ford Coppol…"
"""The Dark Knight""",2008,9.0,"""Action,Crime,Drama""","""PG-13""","""2h 32m""","""Why So Serious?""","""Christian Bale,Heath Ledger,Aa…","""Christopher Nolan""","""Jonathan Nolan,Christopher Nol…"
"""The Godfather Part II""",1974,9.0,"""Crime,Drama""","""R""","""3h 22m""","""All the power on earth can't c…","""Al Pacino,Robert De Niro,Rober…","""Francis Ford Coppola""","""Francis Ford Coppola,Mario Puz…"
"""12 Angry Men""",1957,9.0,"""Crime,Drama""","""Approved""","""1h 36m""","""Life Is In Their Hands -- Deat…","""Henry Fonda,Lee J. Cobb,Martin…","""Sidney Lumet""","""Reginald Rose"""


#### Converting runtime to minutes

In [8]:
def to_minutes(time_str):
    mins = 0
    parts = time_str.split()
    
    for part in parts:
        if part[-1] == 'h':
            mins += int(part[:-1]) * 60
        elif part[-1] == 'm':
            mins += int(part[:-1])
    
    return mins

In [9]:
mins = df["run_time"].map_elements(to_minutes, return_dtype=pl.Int64)
df = df.with_columns(mins)

df.head()

title,year,rating,genre,certificate,run_time,tagline,casts,directors,writers
str,i64,f64,str,str,i64,str,str,str,str
"""The Shawshank Redemption""",1994,9.3,"""Drama""","""R""",142,"""Fear can hold you prisoner. Ho…","""Tim Robbins,Morgan Freeman,Bob…","""Frank Darabont""","""Stephen King,Frank Darabont"""
"""The Godfather""",1972,9.2,"""Crime,Drama""","""R""",175,"""An offer you can't refuse.""","""Marlon Brando,Al Pacino,James …","""Francis Ford Coppola""","""Mario Puzo,Francis Ford Coppol…"
"""The Dark Knight""",2008,9.0,"""Action,Crime,Drama""","""PG-13""",152,"""Why So Serious?""","""Christian Bale,Heath Ledger,Aa…","""Christopher Nolan""","""Jonathan Nolan,Christopher Nol…"
"""The Godfather Part II""",1974,9.0,"""Crime,Drama""","""R""",202,"""All the power on earth can't c…","""Al Pacino,Robert De Niro,Rober…","""Francis Ford Coppola""","""Francis Ford Coppola,Mario Puz…"
"""12 Angry Men""",1957,9.0,"""Crime,Drama""","""Approved""",96,"""Life Is In Their Hands -- Deat…","""Henry Fonda,Lee J. Cobb,Martin…","""Sidney Lumet""","""Reginald Rose"""


#### Importing environment variables

In [10]:
neo4j_url = getenv("NEO4J_URL")
neo4j_user = getenv("NEO4J_USER")
neo4j_password = getenv("NEO4J_PW")
neo4j_database = getenv("NEO4J_DB")

#### Neo4J

In [11]:
driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

if driver.verify_authentication():
    print("Authentication verified")

driver.verify_connectivity()
print("Connection verified")

Authentication verified
Connection verified


#### 3 types of nodes - Movie, Genre, Person
#### Person can be - Actor, Director, Writer or a combination of these
#### Relationships are -
- Person - [ WROTE / ACTED_IN / DIRECTED ] -> Movie
- Movie - [ BELONGS_TO ] -> Genre

In [12]:
def create_graph(df, driver, db, limit):
    with driver.session(database=db) as session:
        for row in df.iter_rows(named=True):
            session.run(
                """
                MERGE (m:Movie {title: $title, year: $year, rating: $rating, certificate: $certificate, run_time: $run_time, tagline: $tagline})
                """,
                title=row['title'],
                year=row['year'],
                rating=row['rating'],
                certificate=row['certificate'],
                run_time=row['run_time'],
                tagline=row['tagline']
            )
            
            actors = row['casts'].split(',')[:limit]
            for actor in actors:
                session.run(
                    """
                    MERGE (p:Person {name: $name})
                    SET p:Actor
                    WITH p
                    MATCH (m:Movie {title: $title})
                    MERGE (p)-[:ACTED_IN]->(m)
                    """,
                    name=actor.strip(),
                    title=row['title']
                )
                
            directors = row['directors'].split(',')[:limit]
            for director in directors:
                session.run(
                    """
                    MERGE (p:Person {name: $name})
                    SET p:Director
                    WITH p
                    MATCH (m:Movie {title: $title})
                    MERGE (p)-[:DIRECTED]->(m)
                    """,
                    name=director.strip(),
                    title=row['title']
                )
                
            writers = row['writers'].split(',')[:limit]
            for writer in writers:
                session.run(
                    """
                    MERGE (p:Person {name: $name})
                    SET p:Writer
                    WITH p
                    MATCH (m:Movie {title: $title})
                    MERGE (p)-[:WROTE]->(m)
                    """,
                    name=writer.strip(),
                    title=row['title']
                )

            genres = row['genre'].split(',')[:limit]
            for genre in genres:
                session.run(
                    """
                    MERGE (g:Genre {name: $name})
                    WITH g
                    MATCH (m:Movie {title: $title})
                    MERGE (m)-[:BELONGS_TO]->(g)
                    """,
                    name=genre.strip(),
                    title=row['title']
                )


Setting the limit parameter to 3, as default, ensures only first 3 actors, directors, writers and genres are considered for a movie.
You can increase / decrease the limit.

In [13]:
create_graph(df, driver, neo4j_database, 3)

In [14]:
driver.close()

- To view the network graph, in the Neo4j browser, run
``` cypher
MATCH (n) RETURN n
```

- To delete the network graph, in the Neo4j browser, run
``` cypher
MATCH (n) DETACH DELETE n
```